# <div align="center" style="color: #ff5733;">711 Bad Rate Check</div>

In [24]:
# %% [markdown]
# # Jupyter Notebook Loading Header
#
# This is a custom loading header for Jupyter Notebooks in Visual Studio Code.
# It includes common imports and settings to get you started quickly.

# %% [markdown]
## Import Libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from google.cloud import bigquery
import os
path = r'C:\Users\DwaipayanChakroborti\AppData\Roaming\gcloud\legacy_credentials\dchakroborti@tonikbank.com\adc.json'
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = path
client = bigquery.Client(project='prj-prod-dataplatform')

# %% [markdown]
## Configure Settings
# Set options or configurations as needed
# Example: pd.set_option('display.max_columns', None)

# Split by Disbursement Date

## Train, Validation and Test Set for All Product with Unique LoanAccountNumber Count

In [25]:
sq = """with mb as
(select customer_id, right(new_mobile_number, 10) mobile from `risk_credit_mis.customer_contact_details`
union all
select customer_id, right(old_mobile_number, 10) mobile from `risk_credit_mis.customer_contact_details` where old_mobile_number is not null
),
mb1
as
(select distinct customer_id, mobile from mb),
mb2
as
(select *, row_number() over(partition by customer_id order by customer_id) custrank from mb1),
delqb as
(Select lmt.customerId, a1.loanAccountNumber, lmt.disbursementDateTime,
row_number() over(partition by lmt.customerId order by disbursementDateTime) rnk
, case when a1.obs_min_inst_def30 >= 3 then lmt.customerId end obsfstpd30
, case when a1.min_inst_def30 in (1,2,3) then lmt.customerId end fstpd30dev
, case when a1.min_inst_def30 in (1) then lmt.customerId end fpd30dev
, case when a1.min_inst_def30 > 0 then lmt.customerId end Thirtyplus
, case when a1.obs_min_inst_def30 >= 1 then lmt.customerId end obsfpd30
from prj-prod-dataplatform.risk_credit_mis.loan_deliquency_data a1
inner join `risk_credit_mis.loan_master_table` lmt on lmt.loanAccountNumber = a1.loanAccountNumber
-- where new_loan_type in ('Quick', 'Flex')
)
,delqb1
as
(select customerId,loanAccountNumber, max(disbursementDateTime) disbursementDateTime,
max(case when obsfstpd30 is not null and fstpd30dev is not null then 1 else 0 end) fstpd30def,
max(case when obsfstpd30 is not null then 1 else 0 end) obsfstpd30,
max(case when fpd30dev is not null then 1 else 0 end) FPD30,
max(case when Thirtyplus is not null then 1 else 0 end) Thirtyplus,
max(case when obsfpd30 is not null then 1 else 0 end) obsfpd30
from delqb 
where obsfstpd30 is not null
-- and rnk = 1
group by 1,2
),
base as
(select * from
`prj-prod-dataplatform.manual_source_extracts.partner_711_payment_transcation_backtest_20240125` a1
inner join mb2 on cast(mb2.mobile as numeric) = cast(a1.mobile_num as numeric)
inner join delqb1 on delqb1.customerId = cast(mb2.customer_id as numeric)
)
--select format_date('%Y-%m', date_trunc(disbursementDateTime, day)) month, count(distinct loanAccountNumber) cntloan ,   count(distinct case when obsfstpd30 = 1 then loanAccountNumber end) obsfstpd30 from base group by 1 order by 1
select case when date_trunc(disbursementDateTime, day) between '2024-02-01' and '2024-03-31' then 'Test' 
when date_trunc(disbursementDateTime, day) between '2024-01-01' and '2024-01-31' then 'Validation'
when date_trunc(disbursementDateTime, day) <= '2023-12-31' then 'Train' else cast(date_trunc(disbursementDateTime, day) as string) end Dataset_type
, min(format_date('%Y-%m-%d',date_trunc(disbursementDateTime, day))) Mindisburseddate
, max(format_date('%Y-%m-%d',date_trunc(disbursementDateTime, day))) Maxdisburseddate
, count(distinct case when fstpd30def = 1 then loanAccountNumber  end ) fstpd30
, count(distinct case when obsfstpd30 = 1 then loanAccountNumber end) obsfstpd30
, count(distinct case when FPD30 = 1 then loanAccountNumber end) FPD30
, count(distinct case when Thirtyplus = 1 then loanAccountNumber end) Thirtyplus
, count(distinct case when obsfpd30 = 1 then loanAccountNumber end) obsfpd30
from base
group by 1
;"""

In [26]:
df = client.query(sq).to_dataframe(progress_bar_type='tqdm')

Job ID c18e3ef9-0c3c-4b3c-9c17-a617b401effb successfully executed: 100%|██████████|
Downloading: 100%|██████████|


In [27]:
df

,Dataset_type,Mindisburseddate,Maxdisburseddate,fstpd30,obsfstpd30,FPD30,Thirtyplus,obsfpd30
0,Train,2021-07-16,2023-12-31,8859,34491,4701,14892,34491
1,Validation,2024-01-01,2024-01-31,55,467,12,78,467
2,Test,2024-02-01,2024-03-08,60,479,21,65,479


In [28]:
df['fstpd30_P'] = round(df['fstpd30']/df['obsfstpd30']*100, 3)
df['fpd30_P'] = round(df['FPD30']/df['obsfpd30']*100, 3)
df['30plus_P'] = round(df['Thirtyplus']/df['obsfpd30']*100, 3)
df


,Dataset_type,Mindisburseddate,Maxdisburseddate,fstpd30,obsfstpd30,FPD30,Thirtyplus,obsfpd30,fstpd30_P,fpd30_P,30plus_P
0,Train,2021-07-16,2023-12-31,8859,34491,4701,14892,34491,25.685,13.63,43.176
1,Validation,2024-01-01,2024-01-31,55,467,12,78,467,11.777,2.57,16.702
2,Test,2024-02-01,2024-03-08,60,479,21,65,479,12.526,4.384,13.57


In [29]:
df.columns

Index(['Dataset_type', 'Mindisburseddate', 'Maxdisburseddate', 'fstpd30',
       'obsfstpd30', 'FPD30', 'Thirtyplus', 'obsfpd30', 'fstpd30_P', 'fpd30_P',
       '30plus_P'],
      dtype='object')

In [30]:
df = df[['Dataset_type', 'Mindisburseddate', 'Maxdisburseddate', 'fstpd30',
       'obsfstpd30', 'fstpd30_P', 'FPD30', 'obsfpd30', 'fpd30_P','Thirtyplus',  '30plus_P']].copy()

In [31]:
df

,Dataset_type,Mindisburseddate,Maxdisburseddate,fstpd30,obsfstpd30,fstpd30_P,FPD30,obsfpd30,fpd30_P,Thirtyplus,30plus_P
0,Train,2021-07-16,2023-12-31,8859,34491,25.685,4701,34491,13.63,14892,43.176
1,Validation,2024-01-01,2024-01-31,55,467,11.777,12,467,2.57,78,16.702
2,Test,2024-02-01,2024-03-08,60,479,12.526,21,479,4.384,65,13.57


In [32]:
from openpyxl import Workbook
from openpyxl.styles import Alignment
from openpyxl import load_workbook

In [33]:
# Create a Workbook object
wb = Workbook()

# Create a sheet and select it
sheet = wb.active
sheet.title = 'Data'

# Calculate the number of columns in the DataFrame
num_columns = len(df.columns)

# Merge cells in row 1
merge_range = f'A1:{chr(ord("A") + num_columns - 1)}1'
sheet.merge_cells(merge_range)

# Set value across merged cells in row 1
merged_cell = sheet[merge_range.split(':')[0]]
merged_cell.value = 'Loan count for all Product'

# Align text to center
merged_cell.alignment = Alignment(horizontal='center', vertical='center')

# Write headers to Excel in row 2
sheet.append(df.columns.tolist())  # Column headers in row 2

# Write the DataFrame data to Excel from row 3 onwards
for row in df.itertuples(index=False, name=None):
    sheet.append(row)

# Save the workbook
wb.save(r"C:\Users\DwaipayanChakroborti\OneDrive - Tonik Financial Pte Ltd\MyStuff\Biswa\711_Data\Data\711_badrate_Analysis_V2.xlsx")

# Train, Validation and Test Set for Quick and Flex Loans with Unique LoanAccountNumber Count

In [34]:
sq = """with mb as
(select customer_id, right(new_mobile_number, 10) mobile from `risk_credit_mis.customer_contact_details`
union all
select customer_id, right(old_mobile_number, 10) mobile from `risk_credit_mis.customer_contact_details` where old_mobile_number is not null
),
mb1
as
(select distinct customer_id, mobile from mb),
mb2
as
(select *, row_number() over(partition by customer_id order by customer_id) custrank from mb1),
delqb as
(Select lmt.customerId, a1.loanAccountNumber, lmt.disbursementDateTime,
row_number() over(partition by lmt.customerId order by disbursementDateTime) rnk
, case when a1.obs_min_inst_def30 >= 3 then lmt.customerId end obsfstpd30
, case when a1.min_inst_def30 in (1,2,3) then lmt.customerId end fstpd30dev
, case when a1.min_inst_def30 in (1) then lmt.customerId end fpd30dev
, case when a1.min_inst_def30 > 0 then lmt.customerId end Thirtyplus
, case when a1.obs_min_inst_def30 >= 1 then lmt.customerId end obsfpd30
from prj-prod-dataplatform.risk_credit_mis.loan_deliquency_data a1
inner join `risk_credit_mis.loan_master_table` lmt on lmt.loanAccountNumber = a1.loanAccountNumber
where new_loan_type in ('Quick', 'Flex')
)
,delqb1
as
(select customerId,loanAccountNumber, max(disbursementDateTime) disbursementDateTime,
max(case when obsfstpd30 is not null and fstpd30dev is not null then 1 else 0 end) fstpd30def,
max(case when obsfstpd30 is not null then 1 else 0 end) obsfstpd30,
max(case when fpd30dev is not null then 1 else 0 end) FPD30,
max(case when Thirtyplus is not null then 1 else 0 end) Thirtyplus,
max(case when obsfpd30 is not null then 1 else 0 end) obsfpd30
from delqb 
where obsfstpd30 is not null
-- and rnk = 1
group by 1,2
),
base as
(select * from
`prj-prod-dataplatform.manual_source_extracts.partner_711_payment_transcation_backtest_20240125` a1
inner join mb2 on cast(mb2.mobile as numeric) = cast(a1.mobile_num as numeric)
inner join delqb1 on delqb1.customerId = cast(mb2.customer_id as numeric)
)
-- select format_date('%Y-%m', date_trunc(disbursementDateTime, day)) month, count(distinct loanAccountNumber) cntloan from base group by 1 order by 1;
select case when date_trunc(disbursementDateTime, day) between '2024-02-01' and '2024-03-31' then 'Test' 
when date_trunc(disbursementDateTime, day) between '2024-01-01' and '2024-01-31' then 'Validation'
when date_trunc(disbursementDateTime, day) <= '2023-12-31' then 'Train' else cast(date_trunc(disbursementDateTime, day) as string) end Dataset_type
, min(format_date('%Y-%m-%d',date_trunc(disbursementDateTime, day))) Mindisburseddate
, max(format_date('%Y-%m-%d',date_trunc(disbursementDateTime, day))) Maxdisburseddate
, count(distinct case when fstpd30def = 1 then loanAccountNumber  end ) fstpd30
, count(distinct case when obsfstpd30 = 1 then loanAccountNumber end) obsfstpd30
, count(distinct case when FPD30 = 1 then loanAccountNumber end) FPD30
, count(distinct case when Thirtyplus = 1 then loanAccountNumber end) Thirtyplus
, count(distinct case when obsfpd30 = 1 then loanAccountNumber end) obsfpd30
from base
group by 1;"""

In [35]:
df = client.query(sq).to_dataframe(progress_bar_type='tqdm')
df['fstpd30_P'] = round(df['fstpd30']/df['obsfstpd30']*100, 3)
df['fpd30_P'] = round(df['FPD30']/df['obsfpd30']*100, 3)
df['30plus_P'] = round(df['Thirtyplus']/df['obsfpd30']*100, 3)
df = df[['Dataset_type', 'Mindisburseddate', 'Maxdisburseddate', 'fstpd30',
       'obsfstpd30', 'fstpd30_P', 'FPD30', 'obsfpd30', 'fpd30_P','Thirtyplus',  '30plus_P']].copy()
df

Job ID 6f3e2ed1-f586-4169-a603-60b7be80f897 successfully executed: 100%|██████████|
Downloading: 100%|██████████|


,Dataset_type,Mindisburseddate,Maxdisburseddate,fstpd30,obsfstpd30,fstpd30_P,FPD30,obsfpd30,fpd30_P,Thirtyplus,30plus_P
0,Train,2021-07-16,2023-12-31,7225,25299,28.558,3862,25299,15.265,11745,46.425
1,Test,2024-02-01,2024-03-07,17,84,20.238,4,84,4.762,19,22.619
2,Validation,2024-01-03,2024-01-31,7,27,25.926,3,27,11.111,10,37.037


In [37]:
# Load the existing workbook
wb = load_workbook(r"C:\Users\DwaipayanChakroborti\OneDrive - Tonik Financial Pte Ltd\MyStuff\Biswa\711_Data\Data\711_badrate_Analysis_V2.xlsx")

# Select the active sheet
sheet = wb.active

# Find the next available blank row
next_row = sheet.max_row + 2

# Calculate the number of columns in the DataFrame
num_columns = len(df.columns)

# Merge cells in row 1
merge_range = f'A{next_row}:{chr(ord("A") + num_columns - 1)}{next_row}'
sheet.merge_cells(merge_range)

# Set value across merged cells in row 1
merged_cell = sheet[f'A{next_row}']
merged_cell.value = 'Loan count for Quick and Flex'

# Align text to center
merged_cell.alignment = Alignment(horizontal='center', vertical='center')

# Write headers to Excel in row 2 if it's the first write operation
sheet.append(df.columns.tolist())  # Column headers in row 2

# Write the DataFrame data to Excel from the next available row onwards
for row in df.itertuples(index=False, name=None):
    sheet.append(row)
    
# Save the workbook
wb.save(r"C:\Users\DwaipayanChakroborti\OneDrive - Tonik Financial Pte Ltd\MyStuff\Biswa\711_Data\Data\711_badrate_Analysis_V2.xlsx")

# Train, Validation and Test Set for All Products Loans with Unique Customer Count

In [38]:
sq = """with mb as
(select customer_id, right(new_mobile_number, 10) mobile from `risk_credit_mis.customer_contact_details`
union all
select customer_id, right(old_mobile_number, 10) mobile from `risk_credit_mis.customer_contact_details` where old_mobile_number is not null
),
mb1
as
(select distinct customer_id, mobile from mb),
mb2
as
(select *, row_number() over(partition by customer_id order by customer_id) custrank from mb1),
delqb as
(Select lmt.customerId, a1.loanAccountNumber, lmt.disbursementDateTime,
row_number() over(partition by lmt.customerId order by disbursementDateTime) rnk
, case when a1.obs_min_inst_def30 >= 3 then lmt.customerId end obsfstpd30
, case when a1.min_inst_def30 in (1,2,3) then lmt.customerId end fstpd30dev
, case when a1.min_inst_def30 in (1) then lmt.customerId end fpd30dev
, case when a1.min_inst_def30 > 0 then lmt.customerId end Thirtyplus
, case when a1.obs_min_inst_def30 >= 1 then lmt.customerId end obsfpd30
from prj-prod-dataplatform.risk_credit_mis.loan_deliquency_data a1
inner join `risk_credit_mis.loan_master_table` lmt on lmt.loanAccountNumber = a1.loanAccountNumber
-- where new_loan_type in ('Quick', 'Flex')
)
,delqb1
as
(select customerId,loanAccountNumber, max(disbursementDateTime) disbursementDateTime,
max(case when obsfstpd30 is not null and fstpd30dev is not null then 1 else 0 end) fstpd30def,
max(case when obsfstpd30 is not null then 1 else 0 end) obsfstpd30,
max(case when fpd30dev is not null then 1 else 0 end) FPD30,
max(case when Thirtyplus is not null then 1 else 0 end) Thirtyplus,
max(case when obsfpd30 is not null then 1 else 0 end) obsfpd30
from delqb 
where obsfstpd30 is not null
-- and rnk = 1
group by 1,2
),
base as
(select * from
`prj-prod-dataplatform.manual_source_extracts.partner_711_payment_transcation_backtest_20240125` a1
inner join mb2 on cast(mb2.mobile as numeric) = cast(a1.mobile_num as numeric)
inner join delqb1 on delqb1.customerId = cast(mb2.customer_id as numeric)
)
-- select format_date('%Y-%m', date_trunc(disbursementDateTime, day)) month, count(distinct loanAccountNumber) cntloan from base group by 1 order by 1;
select case when date_trunc(disbursementDateTime, day) between '2024-02-01' and '2024-03-31' then 'Test' 
when date_trunc(disbursementDateTime, day) between '2024-01-01' and '2024-01-31' then 'Validation'
when date_trunc(disbursementDateTime, day) <= '2023-12-31' then 'Train' else cast(date_trunc(disbursementDateTime, day) as string) end Dataset_type
, min(format_date('%Y-%m-%d',date_trunc(disbursementDateTime, day))) Mindisburseddate
, max(format_date('%Y-%m-%d',date_trunc(disbursementDateTime, day))) Maxdisburseddate
, count(distinct case when fstpd30def = 1 then customerId  end ) fstpd30
, count(distinct case when obsfstpd30 = 1 then customerId end) obsfstpd30
, count(distinct case when FPD30 = 1 then customerId end) FPD30
, count(distinct case when Thirtyplus = 1 then customerId end) Thirtyplus
, count(distinct case when obsfpd30 = 1 then customerId end) obsfpd30
from base
group by 1;"""

In [39]:
df = client.query(sq).to_dataframe(progress_bar_type='tqdm')
df['fstpd30_P'] = round(df['fstpd30']/df['obsfstpd30']*100, 3)
df['fpd30_P'] = round(df['FPD30']/df['obsfpd30']*100, 3)
df['30plus_P'] = round(df['Thirtyplus']/df['obsfpd30']*100, 3)
df = df[['Dataset_type', 'Mindisburseddate', 'Maxdisburseddate', 'fstpd30',
       'obsfstpd30', 'fstpd30_P', 'FPD30', 'obsfpd30', 'fpd30_P','Thirtyplus',  '30plus_P']].copy()

Job ID 2dd6cb33-a45e-4394-b1a2-959c99a7c61e successfully executed: 100%|██████████|
Downloading: 100%|██████████|


In [40]:
df

,Dataset_type,Mindisburseddate,Maxdisburseddate,fstpd30,obsfstpd30,fstpd30_P,FPD30,obsfpd30,fpd30_P,Thirtyplus,30plus_P
0,Train,2021-07-16,2023-12-31,8859,28601,30.974,4701,28601,16.436,14892,52.068
1,Validation,2024-01-01,2024-01-31,55,467,11.777,12,467,2.57,78,16.702
2,Test,2024-02-01,2024-03-08,60,477,12.579,21,477,4.403,65,13.627


In [41]:
# Load the existing workbook
wb = load_workbook(r"C:\Users\DwaipayanChakroborti\OneDrive - Tonik Financial Pte Ltd\MyStuff\Biswa\711_Data\Data\711_badrate_Analysis_V2.xlsx")

# Select the active sheet
sheet = wb.active

# Find the next available blank row
next_row = sheet.max_row + 2

# Calculate the number of columns in the DataFrame
num_columns = len(df.columns)

# Merge cells in row 1
merge_range = f'A{next_row}:{chr(ord("A") + num_columns - 1)}{next_row}'
sheet.merge_cells(merge_range)

# Set value across merged cells in row 1
merged_cell = sheet[f'A{next_row}']
merged_cell.value = 'Unique Customer count for All Products'

# Align text to center
merged_cell.alignment = Alignment(horizontal='center', vertical='center')

# Write headers to Excel in row 2 if it's the first write operation
sheet.append(df.columns.tolist())  # Column headers in row 2

# Write the DataFrame data to Excel from the next available row onwards
for row in df.itertuples(index=False, name=None):
    sheet.append(row)
    
# Save the workbook
wb.save(r"C:\Users\DwaipayanChakroborti\OneDrive - Tonik Financial Pte Ltd\MyStuff\Biswa\711_Data\Data\711_badrate_Analysis_V2.xlsx")

# Train, Validation and Test Set for Quick and Flex Loans with Unique Customer Count

In [42]:
sq = """with mb as
(select customer_id, right(new_mobile_number, 10) mobile from `risk_credit_mis.customer_contact_details`
union all
select customer_id, right(old_mobile_number, 10) mobile from `risk_credit_mis.customer_contact_details` where old_mobile_number is not null
),
mb1
as
(select distinct customer_id, mobile from mb),
mb2
as
(select *, row_number() over(partition by customer_id order by customer_id) custrank from mb1),
delqb as
(Select lmt.customerId, a1.loanAccountNumber, lmt.disbursementDateTime,
row_number() over(partition by lmt.customerId order by disbursementDateTime) rnk
, case when a1.obs_min_inst_def30 >= 3 then lmt.customerId end obsfstpd30
, case when a1.min_inst_def30 in (1,2,3) then lmt.customerId end fstpd30dev
, case when a1.min_inst_def30 in (1) then lmt.customerId end fpd30dev
, case when a1.min_inst_def30 > 0 then lmt.customerId end Thirtyplus
, case when a1.obs_min_inst_def30 >= 1 then lmt.customerId end obsfpd30
from prj-prod-dataplatform.risk_credit_mis.loan_deliquency_data a1
inner join `risk_credit_mis.loan_master_table` lmt on lmt.loanAccountNumber = a1.loanAccountNumber
where new_loan_type in ('Quick', 'Flex')
)
,delqb1
as
(select customerId,loanAccountNumber, max(disbursementDateTime) disbursementDateTime,
max(case when obsfstpd30 is not null and fstpd30dev is not null then 1 else 0 end) fstpd30def,
max(case when obsfstpd30 is not null then 1 else 0 end) obsfstpd30,
max(case when fpd30dev is not null then 1 else 0 end) FPD30,
max(case when Thirtyplus is not null then 1 else 0 end) Thirtyplus,
max(case when obsfpd30 is not null then 1 else 0 end) obsfpd30
from delqb 
where obsfstpd30 is not null
-- and rnk = 1
group by 1,2
),
base as
(select * from
`prj-prod-dataplatform.manual_source_extracts.partner_711_payment_transcation_backtest_20240125` a1
inner join mb2 on cast(mb2.mobile as numeric) = cast(a1.mobile_num as numeric)
inner join delqb1 on delqb1.customerId = cast(mb2.customer_id as numeric)
)
-- select format_date('%Y-%m', date_trunc(disbursementDateTime, day)) month, count(distinct loanAccountNumber) cntloan from base group by 1 order by 1;
select case when date_trunc(disbursementDateTime, day) between '2024-02-01' and '2024-03-31' then 'Test' 
when date_trunc(disbursementDateTime, day) between '2024-01-01' and '2024-01-31' then 'Validation'
when date_trunc(disbursementDateTime, day) <= '2023-12-31' then 'Train' else cast(date_trunc(disbursementDateTime, day) as string) end Dataset_type
, min(format_date('%Y-%m-%d',date_trunc(disbursementDateTime, day))) Mindisburseddate
, max(format_date('%Y-%m-%d',date_trunc(disbursementDateTime, day))) Maxdisburseddate
, count(distinct case when fstpd30def = 1 then customerId  end ) fstpd30
, count(distinct case when obsfstpd30 = 1 then customerId end) obsfstpd30
, count(distinct case when FPD30 = 1 then customerId end) FPD30
, count(distinct case when Thirtyplus = 1 then customerId end) Thirtyplus
, count(distinct case when obsfpd30 = 1 then customerId end) obsfpd30
from base
group by 1;"""

In [43]:
df = client.query(sq).to_dataframe(progress_bar_type='tqdm')
df['fstpd30_P'] = round(df['fstpd30']/df['obsfstpd30']*100, 3)
df['fpd30_P'] = round(df['FPD30']/df['obsfpd30']*100, 3)
df['30plus_P'] = round(df['Thirtyplus']/df['obsfpd30']*100, 3)
df = df[['Dataset_type', 'Mindisburseddate', 'Maxdisburseddate', 'fstpd30',
       'obsfstpd30', 'fstpd30_P', 'FPD30', 'obsfpd30', 'fpd30_P','Thirtyplus',  '30plus_P']].copy()
df

Job ID ca127e77-3a1b-46ba-bc70-cab92f7eff50 successfully executed: 100%|██████████|
Downloading: 100%|██████████|


,Dataset_type,Mindisburseddate,Maxdisburseddate,fstpd30,obsfstpd30,fstpd30_P,FPD30,obsfpd30,fpd30_P,Thirtyplus,30plus_P
0,Train,2021-07-16,2023-12-31,7225,23217,31.119,3862,23217,16.634,11745,50.588
1,Test,2024-02-01,2024-03-07,17,83,20.482,4,83,4.819,19,22.892
2,Validation,2024-01-03,2024-01-31,7,27,25.926,3,27,11.111,10,37.037


In [44]:
# Load the existing workbook
wb = load_workbook(r"C:\Users\DwaipayanChakroborti\OneDrive - Tonik Financial Pte Ltd\MyStuff\Biswa\711_Data\Data\711_badrate_Analysis_V2.xlsx")

# Select the active sheet
sheet = wb.active

# Find the next available blank row
next_row = sheet.max_row + 2

# Calculate the number of columns in the DataFrame
num_columns = len(df.columns)

# Merge cells in row 1
merge_range = f'A{next_row}:{chr(ord("A") + num_columns - 1)}{next_row}'
sheet.merge_cells(merge_range)

# Set value across merged cells in row 1
merged_cell = sheet[f'A{next_row}']
merged_cell.value = 'Unique Customer count for Quick and Flex'

# Align text to center
merged_cell.alignment = Alignment(horizontal='center', vertical='center')

# Write headers to Excel in row 2 if it's the first write operation
if next_row == 2:
    sheet.append(df.columns.tolist())  # Column headers in row 2

# Write the DataFrame data to Excel from the next available row onwards
for row in df.itertuples(index=False, name=None):
    sheet.append(row)
    
# Save the workbook
wb.save(r"C:\Users\DwaipayanChakroborti\OneDrive - Tonik Financial Pte Ltd\MyStuff\Biswa\711_Data\Data\711_badrate_Analysis_V2.xlsx")

# Train, Validation and Test Set for All Products with Unique Loan Count with Data Split on Txn_dtime from payment transcation table

In [ ]:
sq = """


with mb as
(select customer_id, right(new_mobile_number, 10) mobile from `risk_credit_mis.customer_contact_details`
union all
select customer_id, right(old_mobile_number, 10) mobile from `risk_credit_mis.customer_contact_details` where old_mobile_number is not null
),
mb1
as
(select distinct customer_id, mobile from mb),
mb2
as
(select *, row_number() over(partition by customer_id order by customer_id) custrank from mb1),
delqb as
(Select lmt.customerId, a1.loanAccountNumber, lmt.disbursementDateTime,
row_number() over(partition by lmt.customerId order by disbursementDateTime) rnk
, case when a1.obs_min_inst_def30 >= 3 then lmt.customerId end obsfstpd30
, case when a1.min_inst_def30 in (1,2,3) then lmt.customerId end fstpd30dev
, case when a1.min_inst_def30 in (1) then lmt.customerId end fpd30dev
, case when a1.min_inst_def30 > 0 then lmt.customerId end Thirtyplus
, case when a1.obs_min_inst_def30 >= 1 then lmt.customerId end obsfpd30
from prj-prod-dataplatform.risk_credit_mis.loan_deliquency_data a1
inner join `risk_credit_mis.loan_master_table` lmt on lmt.loanAccountNumber = a1.loanAccountNumber
-- where new_loan_type in ('Quick', 'Flex')
)
,delqb1
as
(select customerId,loanAccountNumber, max(disbursementDateTime) disbursementDateTime,
max(case when obsfstpd30 is not null and fstpd30dev is not null then 1 else 0 end) fstpd30def,
max(case when obsfstpd30 is not null then 1 else 0 end) obsfstpd30,
max(case when fpd30dev is not null then 1 else 0 end) FPD30,
max(case when Thirtyplus is not null then 1 else 0 end) Thirtyplus,
max(case when obsfpd30 is not null then 1 else 0 end) obsfpd30
from delqb 
where obsfstpd30 is not null
-- and rnk = 1
group by 1,2
),
base as
(select * from
`prj-prod-dataplatform.manual_source_extracts.partner_711_payment_transcation_backtest_20240125` a1
inner join mb2 on cast(mb2.mobile as numeric) = cast(a1.mobile_num as numeric)
inner join delqb1 on delqb1.customerId = cast(mb2.customer_id as numeric)
)
# select format_date('%Y-%m',date_trunc(txn_datetime, day)) month, count(distinct loanAccountNumber) cntloan ,  count(distinct case when obsfstpd30 = 1 then loanAccountNumber end) obsfstpd30 from base group by 1 order by 1;
# select count(distinct loanAccountNumber)cntloan from base ; 
select case when date_trunc(txn_datetime, day) between '2023-05-01' and '2023-05-31' then 'Test' 
when date_trunc(txn_datetime, day) between '2023-04-16' and '2023-04-30' then 'Validation'
when date_trunc(txn_datetime, day) <= '2023-04-15' then 'Train' else cast(date_trunc(txn_datetime, day) as string) end Dataset_type
, min(format_date('%Y-%m-%d',date_trunc(disbursementDateTime, day))) Mindisburseddate
, max(format_date('%Y-%m-%d',date_trunc(disbursementDateTime, day))) Maxdisburseddate
, count(distinct case when fstpd30def = 1 then loanAccountNumber  end ) fstpd30
, count(distinct case when obsfstpd30 = 1 then loanAccountNumber end) obsfstpd30
, count(distinct case when FPD30 = 1 then loanAccountNumber end) FPD30
, count(distinct case when Thirtyplus = 1 then loanAccountNumber end) Thirtyplus
, count(distinct case when obsfpd30 = 1 then loanAccountNumber end) obsfpd30
from base
group by 1;
"""

In [ ]:
df = client.query(sq).to_dataframe(progress_bar_type='tqdm')
df

In [ ]:
df = client.query(sq).to_dataframe(progress_bar_type='tqdm')
df['fstpd30_P'] = round(df['fstpd30']/df['obsfstpd30']*100, 3)
df['fpd30_P'] = round(df['FPD30']/df['obsfpd30']*100, 3)
df['30plus_P'] = round(df['Thirtyplus']/df['obsfpd30']*100, 3)
df = df[['Dataset_type', 'Mindisburseddate', 'Maxdisburseddate', 'fstpd30',
       'obsfstpd30', 'fstpd30_P', 'FPD30', 'obsfpd30', 'fpd30_P','Thirtyplus',  '30plus_P']].copy()
df

In [ ]:
# Load the existing workbook
wb = load_workbook(r"C:\Users\DwaipayanChakroborti\OneDrive - Tonik Financial Pte Ltd\MyStuff\Biswa\711_Data\Data\711_badrate_Analysis.xlsx")

# Select the active sheet
sheet = wb.active

# Find the next available blank row
next_row = sheet.max_row + 2

# Calculate the number of columns in the DataFrame
num_columns = len(df.columns)

# Merge cells in row 1
merge_range = f'A{next_row}:{chr(ord("A") + num_columns - 1)}{next_row}'
sheet.merge_cells(merge_range)

# Set value across merged cells in row 1
merged_cell = sheet[f'A{next_row}']
merged_cell.value = 'Train, Validation and Test Set for All Products with Unique Loan Count with Data Split on Txn_dtime from payment transcation table'

# Align text to center
merged_cell.alignment = Alignment(horizontal='center', vertical='center')

# Write headers to Excel in row 2 if it's the first write operation
if next_row == 2:
    sheet.append(df.columns.tolist())  # Column headers in row 2

# Write the DataFrame data to Excel from the next available row onwards
for row in df.itertuples(index=False, name=None):
    sheet.append(row)
    
# Save the workbook
wb.save(r"C:\Users\DwaipayanChakroborti\OneDrive - Tonik Financial Pte Ltd\MyStuff\Biswa\711_Data\Data\711_badrate_Analysis.xlsx")

# Train, Validation and Test Set for Quick and Flex with Unique Loan Count with Data Split on Txn_dtime from payment transcation table

In [ ]:
sq = """


with mb as
(select customer_id, right(new_mobile_number, 10) mobile from `risk_credit_mis.customer_contact_details`
union all
select customer_id, right(old_mobile_number, 10) mobile from `risk_credit_mis.customer_contact_details` where old_mobile_number is not null
),
mb1
as
(select distinct customer_id, mobile from mb),
mb2
as
(select *, row_number() over(partition by customer_id order by customer_id) custrank from mb1),
delqb as
(Select lmt.customerId, a1.loanAccountNumber, lmt.disbursementDateTime,
row_number() over(partition by lmt.customerId order by disbursementDateTime) rnk
, case when a1.obs_min_inst_def30 >= 3 then lmt.customerId end obsfstpd30
, case when a1.min_inst_def30 in (1,2,3) then lmt.customerId end fstpd30dev
, case when a1.min_inst_def30 in (1) then lmt.customerId end fpd30dev
, case when a1.min_inst_def30 > 0 then lmt.customerId end Thirtyplus
, case when a1.obs_min_inst_def30 >= 1 then lmt.customerId end obsfpd30
from prj-prod-dataplatform.risk_credit_mis.loan_deliquency_data a1
inner join `risk_credit_mis.loan_master_table` lmt on lmt.loanAccountNumber = a1.loanAccountNumber
where new_loan_type in ('Quick', 'Flex')
)
,delqb1
as
(select customerId,loanAccountNumber, max(disbursementDateTime) disbursementDateTime,
max(case when obsfstpd30 is not null and fstpd30dev is not null then 1 else 0 end) fstpd30def,
max(case when obsfstpd30 is not null then 1 else 0 end) obsfstpd30,
max(case when fpd30dev is not null then 1 else 0 end) FPD30,
max(case when Thirtyplus is not null then 1 else 0 end) Thirtyplus,
max(case when obsfpd30 is not null then 1 else 0 end) obsfpd30
from delqb 
where obsfstpd30 is not null
-- and rnk = 1
group by 1,2
),
base as
(select * from
`prj-prod-dataplatform.manual_source_extracts.partner_711_payment_transcation_backtest_20240125` a1
inner join mb2 on cast(mb2.mobile as numeric) = cast(a1.mobile_num as numeric)
inner join delqb1 on delqb1.customerId = cast(mb2.customer_id as numeric)
)
-- select format_date('%Y-%m', date_trunc(disbursementDateTime, day)) month, count(distinct loanAccountNumber) cntloan from base group by 1 order by 1;
select case when date_trunc(txn_datetime, day) between '2023-05-01' and '2023-05-31' then 'Test' 
when date_trunc(txn_datetime, day) between '2023-04-16' and '2023-04-30' then 'Validation'
when date_trunc(txn_datetime, day) <= '2023-04-15' then 'Train' else cast(date_trunc(txn_datetime, day) as string) end Dataset_type
, min(format_date('%Y-%m-%d',date_trunc(disbursementDateTime, day))) Mindisburseddate
, max(format_date('%Y-%m-%d',date_trunc(disbursementDateTime, day))) Maxdisburseddate
, count(distinct case when fstpd30def = 1 then loanAccountNumber  end ) fstpd30
, count(distinct case when obsfstpd30 = 1 then loanAccountNumber end) obsfstpd30
, count(distinct case when FPD30 = 1 then loanAccountNumber end) FPD30
, count(distinct case when Thirtyplus = 1 then loanAccountNumber end) Thirtyplus
, count(distinct case when obsfpd30 = 1 then loanAccountNumber end) obsfpd30
from base
group by 1;"""

In [ ]:
df = client.query(sq).to_dataframe(progress_bar_type='tqdm')
df['fstpd30_P'] = round(df['fstpd30']/df['obsfstpd30']*100, 3)
df['fpd30_P'] = round(df['FPD30']/df['obsfpd30']*100, 3)
df['30plus_P'] = round(df['Thirtyplus']/df['obsfpd30']*100, 3)
df = df[['Dataset_type', 'Mindisburseddate', 'Maxdisburseddate', 'fstpd30',
       'obsfstpd30', 'fstpd30_P', 'FPD30', 'obsfpd30', 'fpd30_P','Thirtyplus',  '30plus_P']].copy()
df

In [ ]:
# Load the existing workbook
wb = load_workbook(r"C:\Users\DwaipayanChakroborti\OneDrive - Tonik Financial Pte Ltd\MyStuff\Biswa\711_Data\Data\711_badrate_Analysis.xlsx")

# Select the active sheet
sheet = wb.active

# Find the next available blank row
next_row = sheet.max_row + 2

# Calculate the number of columns in the DataFrame
num_columns = len(df.columns)

# Merge cells in row 1
merge_range = f'A{next_row}:{chr(ord("A") + num_columns - 1)}{next_row}'
sheet.merge_cells(merge_range)

# Set value across merged cells in row 1
merged_cell = sheet[f'A{next_row}']
merged_cell.value = 'Train, Validation and Test Set for Quick and Flex with Unique Loan Count with Data Split on Txn_dtime from payment transcation table'

# Align text to center
merged_cell.alignment = Alignment(horizontal='center', vertical='center')

# Write headers to Excel in row 2 if it's the first write operation
if next_row == 2:
    sheet.append(df.columns.tolist())  # Column headers in row 2

# Write the DataFrame data to Excel from the next available row onwards
for row in df.itertuples(index=False, name=None):
    sheet.append(row)
    
# Save the workbook
wb.save(r"C:\Users\DwaipayanChakroborti\OneDrive - Tonik Financial Pte Ltd\MyStuff\Biswa\711_Data\Data\711_badrate_Analysis.xlsx")

# Train, Validation and Test Set for All Product with Unique Customer Count with Data Split on Txn_dtime from payment transcation table

In [ ]:
sq = """


with mb as
(select customer_id, right(new_mobile_number, 10) mobile from `risk_credit_mis.customer_contact_details`
union all
select customer_id, right(old_mobile_number, 10) mobile from `risk_credit_mis.customer_contact_details` where old_mobile_number is not null
),
mb1
as
(select distinct customer_id, mobile from mb),
mb2
as
(select *, row_number() over(partition by customer_id order by customer_id) custrank from mb1),
delqb as
(Select lmt.customerId, a1.loanAccountNumber, lmt.disbursementDateTime,
row_number() over(partition by lmt.customerId order by disbursementDateTime) rnk
, case when a1.obs_min_inst_def30 >= 3 then lmt.customerId end obsfstpd30
, case when a1.min_inst_def30 in (1,2,3) then lmt.customerId end fstpd30dev
, case when a1.min_inst_def30 in (1) then lmt.customerId end fpd30dev
, case when a1.min_inst_def30 > 0 then lmt.customerId end Thirtyplus
, case when a1.obs_min_inst_def30 >= 1 then lmt.customerId end obsfpd30
from prj-prod-dataplatform.risk_credit_mis.loan_deliquency_data a1
inner join `risk_credit_mis.loan_master_table` lmt on lmt.loanAccountNumber = a1.loanAccountNumber
-- where new_loan_type in ('Quick', 'Flex')
)
,delqb1
as
(select customerId,loanAccountNumber, max(disbursementDateTime) disbursementDateTime,
max(case when obsfstpd30 is not null and fstpd30dev is not null then 1 else 0 end) fstpd30def,
max(case when obsfstpd30 is not null then 1 else 0 end) obsfstpd30,
max(case when fpd30dev is not null then 1 else 0 end) FPD30,
max(case when Thirtyplus is not null then 1 else 0 end) Thirtyplus,
max(case when obsfpd30 is not null then 1 else 0 end) obsfpd30
from delqb 
where obsfstpd30 is not null
-- and rnk = 1
group by 1,2
),
base as
(select * from
`prj-prod-dataplatform.manual_source_extracts.partner_711_payment_transcation_backtest_20240125` a1
inner join mb2 on cast(mb2.mobile as numeric) = cast(a1.mobile_num as numeric)
inner join delqb1 on delqb1.customerId = cast(mb2.customer_id as numeric)
)
-- select format_date('%Y-%m', date_trunc(disbursementDateTime, day)) month, count(distinct loanAccountNumber) cntloan from base group by 1 order by 1;
select case when date_trunc(txn_datetime, day) between '2023-05-01' and '2023-05-31' then 'Test' 
when date_trunc(txn_datetime, day) between '2023-04-16' and '2023-04-30' then 'Validation'
when date_trunc(txn_datetime, day) <= '2023-04-15' then 'Train' else cast(date_trunc(txn_datetime, day) as string) end Dataset_type
, min(format_date('%Y-%m-%d',date_trunc(disbursementDateTime, day))) Mindisburseddate
, max(format_date('%Y-%m-%d',date_trunc(disbursementDateTime, day))) Maxdisburseddate
, count(distinct case when fstpd30def = 1 then customerId  end ) fstpd30
, count(distinct case when obsfstpd30 = 1 then customerId end) obsfstpd30
, count(distinct case when FPD30 = 1 then customerId end) FPD30
, count(distinct case when Thirtyplus = 1 then customerId end) Thirtyplus
, count(distinct case when obsfpd30 = 1 then customerId end) obsfpd30
from base
group by 1;"""

In [ ]:
df = client.query(sq).to_dataframe(progress_bar_type='tqdm')
df['fstpd30_P'] = round(df['fstpd30']/df['obsfstpd30']*100, 3)
df['fpd30_P'] = round(df['FPD30']/df['obsfpd30']*100, 3)
df['30plus_P'] = round(df['Thirtyplus']/df['obsfpd30']*100, 3)
df = df[['Dataset_type', 'Mindisburseddate', 'Maxdisburseddate', 'fstpd30',
       'obsfstpd30', 'fstpd30_P', 'FPD30', 'obsfpd30', 'fpd30_P','Thirtyplus',  '30plus_P']].copy()
df

In [ ]:
# Load the existing workbook
wb = load_workbook(r"C:\Users\DwaipayanChakroborti\OneDrive - Tonik Financial Pte Ltd\MyStuff\Biswa\711_Data\Data\711_badrate_Analysis.xlsx")

# Select the active sheet
sheet = wb.active

# Find the next available blank row
next_row = sheet.max_row + 2

# Calculate the number of columns in the DataFrame
num_columns = len(df.columns)

# Merge cells in row 1
merge_range = f'A{next_row}:{chr(ord("A") + num_columns - 1)}{next_row}'
sheet.merge_cells(merge_range)

# Set value across merged cells in row 1
merged_cell = sheet[f'A{next_row}']
merged_cell.value = 'Train, Validation and Test Set for All Product with Unique Customer Count with Data Split on Txn_dtime from payment transcation table'

# Align text to center
merged_cell.alignment = Alignment(horizontal='center', vertical='center')

# Write headers to Excel in row 2 if it's the first write operation
if next_row == 2:
    sheet.append(df.columns.tolist())  # Column headers in row 2

# Write the DataFrame data to Excel from the next available row onwards
for row in df.itertuples(index=False, name=None):
    sheet.append(row)
    
# Save the workbook
wb.save(r"C:\Users\DwaipayanChakroborti\OneDrive - Tonik Financial Pte Ltd\MyStuff\Biswa\711_Data\Data\711_badrate_Analysis.xlsx")

# Train, Validation and Test Set for Quick and Flex with Unique Customer Count with Data Split on Txn_dtime from payment transcation table

In [ ]:
sq = """


with mb as
(select customer_id, right(new_mobile_number, 10) mobile from `risk_credit_mis.customer_contact_details`
union all
select customer_id, right(old_mobile_number, 10) mobile from `risk_credit_mis.customer_contact_details` where old_mobile_number is not null
),
mb1
as
(select distinct customer_id, mobile from mb),
mb2
as
(select *, row_number() over(partition by customer_id order by customer_id) custrank from mb1),
delqb as
(Select lmt.customerId, a1.loanAccountNumber, lmt.disbursementDateTime,
row_number() over(partition by lmt.customerId order by disbursementDateTime) rnk
, case when a1.obs_min_inst_def30 >= 3 then lmt.customerId end obsfstpd30
, case when a1.min_inst_def30 in (1,2,3) then lmt.customerId end fstpd30dev
, case when a1.min_inst_def30 in (1) then lmt.customerId end fpd30dev
, case when a1.min_inst_def30 > 0 then lmt.customerId end Thirtyplus
, case when a1.obs_min_inst_def30 >= 1 then lmt.customerId end obsfpd30
from prj-prod-dataplatform.risk_credit_mis.loan_deliquency_data a1
inner join `risk_credit_mis.loan_master_table` lmt on lmt.loanAccountNumber = a1.loanAccountNumber
where new_loan_type in ('Quick', 'Flex')
)
,delqb1
as
(select customerId,loanAccountNumber, max(disbursementDateTime) disbursementDateTime,
max(case when obsfstpd30 is not null and fstpd30dev is not null then 1 else 0 end) fstpd30def,
max(case when obsfstpd30 is not null then 1 else 0 end) obsfstpd30,
max(case when fpd30dev is not null then 1 else 0 end) FPD30,
max(case when Thirtyplus is not null then 1 else 0 end) Thirtyplus,
max(case when obsfpd30 is not null then 1 else 0 end) obsfpd30
from delqb 
where obsfstpd30 is not null
-- and rnk = 1
group by 1,2
),
base as
(select * from
`prj-prod-dataplatform.manual_source_extracts.partner_711_payment_transcation_backtest_20240125` a1
inner join mb2 on cast(mb2.mobile as numeric) = cast(a1.mobile_num as numeric)
inner join delqb1 on delqb1.customerId = cast(mb2.customer_id as numeric)
)
-- select format_date('%Y-%m', date_trunc(disbursementDateTime, day)) month, count(distinct loanAccountNumber) cntloan from base group by 1 order by 1;
select case when date_trunc(txn_datetime, day) between '2023-05-01' and '2023-05-31' then 'Test' 
when date_trunc(txn_datetime, day) between '2023-04-16' and '2023-04-30' then 'Validation'
when date_trunc(txn_datetime, day) <= '2023-04-15' then 'Train' else cast(date_trunc(txn_datetime, day) as string) end Dataset_type
, min(format_date('%Y-%m-%d',date_trunc(disbursementDateTime, day))) Mindisburseddate
, max(format_date('%Y-%m-%d',date_trunc(disbursementDateTime, day))) Maxdisburseddate
, count(distinct case when fstpd30def = 1 then customerId  end ) fstpd30
, count(distinct case when obsfstpd30 = 1 then customerId end) obsfstpd30
, count(distinct case when FPD30 = 1 then customerId end) FPD30
, count(distinct case when Thirtyplus = 1 then customerId end) Thirtyplus
, count(distinct case when obsfpd30 = 1 then customerId end) obsfpd30
from base
group by 1;"""

In [ ]:
df = client.query(sq).to_dataframe(progress_bar_type='tqdm')
df['fstpd30_P'] = round(df['fstpd30']/df['obsfstpd30']*100, 3)
df['fpd30_P'] = round(df['FPD30']/df['obsfpd30']*100, 3)
df['30plus_P'] = round(df['Thirtyplus']/df['obsfpd30']*100, 3)
df = df[['Dataset_type', 'Mindisburseddate', 'Maxdisburseddate', 'fstpd30',
       'obsfstpd30', 'fstpd30_P', 'FPD30', 'obsfpd30', 'fpd30_P','Thirtyplus',  '30plus_P']].copy()
df

In [ ]:
# Load the existing workbook
wb = load_workbook(r"C:\Users\DwaipayanChakroborti\OneDrive - Tonik Financial Pte Ltd\MyStuff\Biswa\711_Data\Data\711_badrate_Analysis.xlsx")

# Select the active sheet
sheet = wb.active

# Find the next available blank row
next_row = sheet.max_row + 2

# Calculate the number of columns in the DataFrame
num_columns = len(df.columns)

# Merge cells in row 1
merge_range = f'A{next_row}:{chr(ord("A") + num_columns - 1)}{next_row}'
sheet.merge_cells(merge_range)

# Set value across merged cells in row 1
merged_cell = sheet[f'A{next_row}']
merged_cell.value = 'Train, Validation and Test Set for Quick and Flex with Unique Customer Count with Data Split on Txn_dtime from payment transcation table'

# Align text to center
merged_cell.alignment = Alignment(horizontal='center', vertical='center')

# Write headers to Excel in row 2 if it's the first write operation
if next_row == 2:
    sheet.append(df.columns.tolist())  # Column headers in row 2

# Write the DataFrame data to Excel from the next available row onwards
for row in df.itertuples(index=False, name=None):
    sheet.append(row)
    
# Save the workbook
wb.save(r"C:\Users\DwaipayanChakroborti\OneDrive - Tonik Financial Pte Ltd\MyStuff\Biswa\711_Data\Data\711_badrate_Analysis.xlsx")